Imports

In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd
from RealtimeTracking import *
import gc

Load the model

In [2]:
plate_det_model_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/models/YOLOv8/yolov8n_90e_cust/runs/detect/train4/weights/best.pt'
testvideo_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/video_data/testvideo_highway_mp4.mp4'

vehicle_det_model=YOLO('yolov8n.pt')
vehicle_ids = [2,3,5,7]
plate_det_model = YOLO(plate_det_model_path)

Capture on a video

In [3]:
def PlateToVehicle(det_license_plate,det_vehicles):
    lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_license_plate
    found=False
    for i in range(len(det_vehicles)):
        v_x1, v_y1, v_x2, v_y2, v_id = det_vehicles[i]
        #print(v_id)
        #inside
        if lp_x1 > v_x1 and lp_y1 > v_y1 and lp_x2 < v_x2 and lp_y2 < v_y2:
            v_index=i
            found=True
            break
    if found:
        return det_vehicles[v_index]
    return -1,-1,-1,-1,-1

#def get_car(license_plate, vehicle_track_ids):
#    x1, y1, x2, y2, score, class_id = license_plate
#
#    foundIt = False
#    for j in range(len(vehicle_track_ids)):
#        xcar1, ycar1, xcar2, ycar2, car_id = vehicle_track_ids[j]
#        print(car_id)
#
#        if x1 > xcar1 and y1 > ycar1 and x2 < xcar2 and y2 < ycar2:
#            car_indx = j
#            foundIt = True
#            break
#
#    if foundIt:
#        return vehicle_track_ids[car_indx]
#
#    return -1, -1, -1, -1, -1

def lpPrep(license_plate):
    lp_gray = cv2.cvtColor(license_plate,cv2.COLOR_BGR2GRAY)
    retval,lp_threshold = cv2.threshold(lp_gray,64,255,cv2.THRESH_BINARY_INV)
    return lp_threshold

def lpRead(license_plate):
    lp_Text='asd012'
    lp_Conf_score = 1
    return lp_Text, lp_Conf_score

In [ ]:
gc.collect()

In [ ]:
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []

# load using Sort
video_capture = cv2.VideoCapture(testvideo_path)


mot_trckr = Sort()
results = {}
ret = True
frame_indexer=0
while ret:
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
    
    ret, frame = video_capture.read()
    
    if ret and frame_indexer<10:
        results[frame_indexer] = {}
        #det
        det_objs = vehicle_det_model(frame)[0]
        vehicles=[]
        for det_obj in det_objs.boxes.data.tolist():
            obj_x1, obj_y1, obj_x2, obj_y2, obj_score, obj_class_id = det_obj
            if int(obj_class_id) in vehicle_ids:
                vehicles.append([obj_x1, obj_y1, obj_x2, obj_y2, obj_score])
                print(obj_class_id)
        
        track_ids = mot_trckr.update(np.asarray(vehicles))
        #print(track_ids)
        #plates=[]
        det_lps = plate_det_model(frame)[0]
        for det_lp in det_lps.boxes.data.tolist():
            lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_lp
            
            #assign license plate to vehicle
            vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2,vehicle_id = PlateToVehicle(det_lp,track_ids)
            if vehicle_id is not -1:
                #crop
                license_plate = frame[int(lp_y1):int(lp_y2),int(lp_x1):int(lp_x2),:]
                prepared = lpPrep(license_plate)
                #cv2.imshow('lp',license_plate)
                #cv2.imshow('lpprep',prepared)
                #cv2.waitKey(0)

                lp_text, lp_text_confscore = lpRead(prepared)
                if lp_text is not None:
                    #results[indexer][vehicle_id] = {'vehicle': {'bbox': [vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2]},
                    #                                'license_plate': {'bbox': [x1, y1, x2, y2],
                    #                                                  'text':lp_text,
                    #                                                  'bbox_confscore':score,
                    #                                                  'text_confscore':lp_text_confscore
                    #                                                 }}
                    df_rows.append({'fr_number':frame_indexer,
                                    'v_id':int(obj_class_id),
                                    'v_bbox':[vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2],
                                    'lp_bbox':[lp_x1, lp_y1, lp_x2, lp_y2],
                                    'lp_bbox_score':lp_score,
                                    'lp':lp_text,
                                    'lp_score':lp_text_confscore})
                
            
    
    frame_indexer+=1

In [4]:
def write_csv(results, output_path):
    """
    Write the results to a CSV file.

    Args:
        results (dict): Dictionary containing the results.
        output_path (str): Path to the output CSV file.
    """
    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{}\n'.format('frame_nmr', 'car_id', 'car_bbox',
                                                'license_plate_bbox', 'license_plate_bbox_score', 'license_number',
                                                'license_number_score'))

        for frame_nmr in results.keys():
            for car_id in results[frame_nmr].keys():
                print(results[frame_nmr][car_id])
                if 'car' in results[frame_nmr][car_id].keys() and \
                   'license_plate' in results[frame_nmr][car_id].keys() and \
                   'text' in results[frame_nmr][car_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{}\n'.format(frame_nmr,
                                                            car_id,
                                                            '[{} {} {} {}]'.format(
                                                                results[frame_nmr][car_id]['car']['bbox'][0],
                                                                results[frame_nmr][car_id]['car']['bbox'][1],
                                                                results[frame_nmr][car_id]['car']['bbox'][2],
                                                                results[frame_nmr][car_id]['car']['bbox'][3]),
                                                            '[{} {} {} {}]'.format(
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][0],
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][1],
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][2],
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][3]),
                                                            results[frame_nmr][car_id]['license_plate']['bbox_score'],
                                                            results[frame_nmr][car_id]['license_plate']['text'],
                                                            results[frame_nmr][car_id]['license_plate']['text_score'])
                            )
        f.close()



In [5]:
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []
vehicles = [2, 3, 5, 7]
video_capture = cv2.VideoCapture(testvideo_path)

results = {}
mot_tracker = Sort()

# read frames
frame_nmr = -1
ret = True
while ret and frame_nmr<10:
    frame_nmr += 1
    ret, frame = video_capture.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = vehicle_det_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, det_class_id = detection
            if int(det_class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])
                print(det_class_id)

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = plate_det_model(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = PlateToVehicle(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = lpRead(license_plate_crop_thresh)

                if license_plate_text is not None:
                    df_rows.append({'fr_number':frame_nmr,
                                    'v_id':int(car_id),
                                    'v_bbox':[xcar1, ycar1, xcar2, ycar2,],
                                    'lp_bbox':[x1, y1, x2, y2],
                                    'lp_bbox_score':score,
                                    'lp':license_plate_text,
                                    'lp_score':license_plate_text_score})
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
frame_nmr +=1

[ WARN:0@8.073] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@8.073] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@8.073] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@8.073] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_16645483

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
7.0
2.0
2.0
2.0
7.0
2.0
2.0
2.0
2.0
2.0
2.0
5.0
2.0
2.0
5.0
2.0
2.0



0: 480x640 4 licenses, 126.9ms
Speed: 4.0ms preprocess, 126.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 23 cars, 2 buss, 2 trucks, 160.3ms
Speed: 3.7ms preprocess, 160.3ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)



2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
7.0
2.0
2.0
2.0
7.0
2.0
2.0
2.0
2.0
5.0
2.0
2.0
2.0
2.0
5.0
2.0
2.0


0: 480x640 4 licenses, 183.3ms
Speed: 4.5ms preprocess, 183.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 23 cars, 2 buss, 2 trucks, 145.4ms
Speed: 5.2ms preprocess, 145.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 licenses, 106.6ms
Speed: 3.9ms preprocess, 106.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)



...

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
7.0
7.0
2.0
2.0
2.0
2.0
2.0
2.0
5.0
2.0
2.0
2.0
2.0
5.0


0: 480x640 21 cars, 2 buss, 2 trucks, 86.4ms
Speed: 3.3ms preprocess, 86.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 licenses, 90.9ms
Speed: 3.2ms preprocess, 90.9ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)



2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
7.0
7.0
2.0
2.0
2.0
2.0
2.0
2.0
5.0
2.0
2.0
2.0
2.0
5.0


0: 480x640 21 cars, 2 buss, 2 trucks, 94.0ms
Speed: 3.2ms preprocess, 94.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 licenses, 175.4ms
Speed: 3.2ms preprocess, 175.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)


2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
7.0
7.0
2.0
2.0
2.0
2.0
2.0
2.0
5.0
2.0
2.0
2.0
2.0
5.0


In [6]:
out = pd.DataFrame(df_rows,columns=df_cols)
out

,fr_number,v_id,v_bbox,lp_bbox,lp_bbox_score,lp,lp_score
0,0,4,"[252.07601928710938, 549.6202392578125, 575.88...","[355.4010314941406, 719.619140625, 461.6276550...",0.467115,asd012,1
1,0,1,"[858.7146606445312, 616.114501953125, 1221.752...","[986.8820190429688, 815.9974365234375, 1114.75...",0.333886,asd012,1
2,0,2,"[980.7061767578125, 900.4325561523438, 1461.83...","[1159.7911376953125, 1149.2432861328125, 1293....",0.308768,asd012,1
3,1,4,"[252.11460278559866, 549.6452530102526, 575.88...","[355.40081787109375, 719.6344604492188, 461.58...",0.467120,asd012,1
4,1,1,"[858.4844545261479, 616.0321937067034, 1221.94...","[986.96044921875, 816.00048828125, 1114.767578...",0.333503,asd012,1
5,1,2,"[980.7313426363543, 900.4754747630882, 1461.75...","[1159.7852783203125, 1149.1611328125, 1293.970...",0.307354,asd012,1
6,2,4,"[252.0693792846115, 549.6147175445177, 575.898...","[355.51629638671875, 719.6675415039062, 461.61...",0.467570,asd012,1
7,2,1,"[858.3735940702186, 616.0541602149467, 1222.04...","[987.1072998046875, 815.9970703125, 1114.69360...",0.332931,asd012,1
8,2,2,"[980.8782300976031, 900.4811413778303, 1461.93...","[1160.0576171875, 1149.122314453125, 1293.8375...",0.307911,asd012,1
9,3,4,"[251.91572972634643, 549.567868758543, 575.840...","[355.5184326171875, 719.661376953125, 461.6175...",0.467799,asd012,1


In [7]:
write_csv(results,'d_log.csv')

{'car': {'bbox': [252.07601928710938, 549.6202392578125, 575.8872680664062, 830.0827026367188]}, 'license_plate': {'bbox': [355.4010314941406, 719.619140625, 461.6276550292969, 760.1126708984375], 'text': 'asd012', 'bbox_score': 0.4671148955821991, 'text_score': 1}}
{'car': {'bbox': [858.7146606445312, 616.114501953125, 1221.7525634765625, 915.2742309570312]}, 'license_plate': {'bbox': [986.8820190429688, 815.9974365234375, 1114.755615234375, 849.2803955078125], 'text': 'asd012', 'bbox_score': 0.33388593792915344, 'text_score': 1}}
{'car': {'bbox': [980.7061767578125, 900.4325561523438, 1461.8397216796875, 1205.5289306640625]}, 'license_plate': {'bbox': [1159.7911376953125, 1149.2432861328125, 1293.81201171875, 1204.4459228515625], 'text': 'asd012', 'bbox_score': 0.3087675869464874, 'text_score': 1}}
{'car': {'bbox': [252.11460278559866, 549.6452530102526, 575.8846459415513, 830.064280022764]}, 'license_plate': {'bbox': [355.40081787109375, 719.6344604492188, 461.5819091796875, 760.101

In [8]:
out.to_csv('df_log.csv')